# Monte-Carlo and Weighted Covariance Matrices

This notebook demonstrates how to simulate correlated stock returns, compute portfolio variance, and estimate portfolio risk using Monte Carlo methods. Specifically, we will (1) Simulate correlated stock returns using a multivariate normal distribution (2) Calculate portfolio variance and standard deviation, accounting for weights and covariances and (3) Use Monte Carlo simulations to approximate expected portfolio returns and risk.  

The goal is to illustrate key concepts in portfolio diversification and risk management through practical Python examples.

## Simulating Two Correlated Stocks

We simulate two correlated stocks —  whose movements are linked either through direct economic relationships (substitutes or complements, i.e., causative correlation) or through shared exposure to broader market trends (non-causative correlation). We denote the return of the i-th stock by the random variable $r_i$. We declare a **mean vector** and **covariance matrix**, the latter given by  
$$\Sigma = [A_{ij}] = \operatorname{Cov}(r_i, r_j)$$
- If $ i = j $, then $\operatorname{Cov}(r_i, r_i) = \operatorname{Var}(r_i) =\sigma_{r_i}^2$ .  
- If $ i \neq j $, then  $
\operatorname{Cov}(r_i, r_j) = \rho_{r_i, r_j} \, \sigma_{r_i}\sigma_{r_j}$, where $ \rho $ is the correlation coefficient (i.e., normalized covariance) and $ \sigma $ represents standard deviation.

In the matrix below, we set  $
\sigma_{r_1} = 0.02, 
\sigma_{r_2} = 0.03, 
\rho = 0.5
$, and fill the covariance matrix with the corresponding values. We then sample the **multivariate normal distribution** — centered at $ \mu $ (the mean vector) — whose elliptical behavior depends on $\Sigma$. We draw 100 samples, corresponding to 100 days of simulated stock return trends.

We then estimate the correlation coefficient and set `rowvar=False` because the variables are stored in columns. Notice that the computed correlation is approximately $ 0.5$.


In [45]:
import numpy as np

np.random.seed(42) 
mu = [0.001, 0.0005]   
cov = [[0.0004, 0.0003],  
       [0.0003, 0.0009]]
returns2 = np.random.multivariate_normal(mean = mu, cov = cov, size = 100)

print("  Stock 1   ", "Stock 2 \n", returns2)
print("Here is a correlation matrix: \n", np.corrcoef(returns2, rowvar=False)) 

  Stock 1    Stock 2 
 [[ 0.00577906  0.0159545 ]
 [ 0.0320793   0.00901382]
 [-0.00561922 -0.00474024]
 [ 0.03380192  0.04138775]
 [ 0.00249142 -0.01692116]
 [-0.01213444 -0.00985504]
 [-0.02360001  0.02064069]
 [-0.03080281 -0.04604571]
 [-0.00827335 -0.03123351]
 [-0.03202595 -0.01637517]
 [ 0.01775939  0.04485654]
 [-0.01886001  0.01220728]
 [-0.00483011 -0.01616016]
 [-0.00926726 -0.03568931]
 [-0.01147309 -0.01505259]
 [ 0.01978812 -0.02973259]
 [-0.01461434  0.00733259]
 [-0.00555538  0.03286932]
 [-0.0247296   0.01999079]
 [-0.01430043 -0.03964347]
 [ 0.01360356  0.02090089]
 [-0.00497473 -0.00082096]
 [-0.02973003 -0.03777158]
 [ 0.01011863 -0.02017883]
 [-0.02001749  0.0225837 ]
 [-0.00018339  0.0125981 ]
 [ 0.00066138 -0.02345327]
 [ 0.02869143  0.0242541 ]
 [-0.01499288 -0.02190216]
 [ 0.01976339  0.00351126]
 [-0.0082644  -0.01222891]
 [-0.03158686 -0.02364464]
 [ 0.0319015   0.01496857]
 [ 0.01465409 -0.00846022]
 [-0.00346298  0.01547178]
 [ 0.02838116  0.00054821]
 [ 0.

## Portfolio Variance

We calculate portfolio variance given by $$\operatorname{Var}(w_1r_1+w_2r_2)=w_1^2\operatorname{Var}(r_1) + 2w_1w_2\operatorname{Cov}(r_1,r_2) + w_2^2\operatorname{Var}(r_2),$$which can also be written $$w^T \cdot \Sigma \cdot w$$
Thus, we get the variance in our linear combination of stocks accounting for individual stock variance and how they move together.

For example, if $w_1 = 0.5, w_2 = 0.5$, then portfolio variance is given by $0.25\operatorname{Var}(r_1)+0.25\operatorname{Var}(r_2)+0.5\operatorname{Covar}(r_1,r_2)$, i.e. individual stock variance and how they move together are weighted equally (but do not necessarily contribute equally, as that would depend on the actual value of variance and covariance). 

Since we set weights $w_1+w_2=1$, the factor on the covariance term $w_1w_2=w_1(1-w_1)$ has min/max at $w_1,w_2 = 0.5$ and $w_1 = 1, w_2 = 0$ or $w_1 = 0, w_2 = 1$. Intuitively, this means that the contribution of the covariance term can be reduced or “eliminated” by weighting one stock much more heavily than the other. 

It is important to note that the weights control the relative contribution of covariance versus individual variance, but not the overall portfolio variance alone. A single stock with inherently high variance may still dominate portfolio risk. To minimize portfolio variance, one must consider both the weights and the individual stock variances and covariances. Moreover, if the covariance is negative, equal weighting may actually reduce overall portfolio variance. This type of logic underpins portfolio diversification, in which stocks are selected to reduce variance at a similar level of expected return $E(w_1r_1+w_2r_2)$.

In [67]:
def portfolioVar(weights, cov):
    return (weights.T @ cov @ weights) 
portfolioVar(np.array([0.2, 0.8]), cov)

0.000688

## Monte-Carlo Methods
We apply the Monte Carlo method — repeating independent experiments to approximate values that are difficult to calculate analytically — to estimate portfolio expected return and variance using the previously defined functions. We simulate portfolio returns for 3 selected stocks over 60 days, repeating the simulation across 100 independent trials.

Note: In the final step, it is also cleaner to set the covariance matrix using cov = np.diag(VarOverDays[x, :]), which treats the stocks as independent and uses their daily variances. We also calculated the average portfolio standard deviation (the typical day-to-day risk), not the variation of standard deviation between simulations. Both measures can provide useful insights.

In [86]:
def monte(simulation, day, stock):
    np.random.seed(42)
    return np.random.normal(0,0.02, size = (simulation,day,stock))



def means(weights, simulation = 100, day = 60, stock = 3):
    array = monte(simulation, day, stock)
    meansOverDays = np.zeros((simulation, stock)) 
    VarOverDays = np.zeros((simulation, stock)) 
    for x in range(simulation):
        for z in range(stock):
            meansOverDays[x, z] = array[x, :, z].mean() #mean for each stock and sim number
            VarOverDays[x, z] = array[x, :, z].var() #var for each stock and sim number
    meansOverStocks = np.zeros(simulation)
    VarOverStocks = np.zeros(simulation)
    for x in range(simulation):
        meansOverStocks[x] = meansOverDays[x, :] @ weights #portfolio mean for each sim
        VarOverStocks[x] = portfolioVar(weights, cov = np.array([[VarOverDays[x, 0], 0, 0], [0.0, VarOverDays[x, 1], 0], [0, 0, VarOverDays[x, 2]]])) #portfolio variance for each sim, assumed no covariance between stocks 
    return meansOverStocks.mean(), np.sqrt(VarOverStocks.mean()) #portfolio mean across all sims, portfolio SD across all sims. 
            
means(np.array([0.2, 0.2, 0.6]))

(6.311506430179842e-05, 0.013095611070084659)